# (02) Daniel

**Motivation**: save raw data (both bold & ca) as matlab files. <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-Ca-fMRI/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-Ca-fMRI/figs')
tmp_dir = os.path.join(git_dir, 'jb-Ca-fMRI/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_Ca-fMRI'))
from figures.fighelper import *
from analysis.final import *
from utils.render import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tqdm.notebook import tqdm
%matplotlib inline
set_style()

In [2]:
from scipy import io as sio

src = '/home/hadi/Documents/Ca-fMRI/processed/norm-global_parcel-columnar_n-512*3'
tgt = '/home/hadi/Desktop/Hadi-SLC'

In [3]:
shapes = collections.defaultdict(list)
for mode in ['bold', 'ca2']:
    path = pjoin(src, mode)
    for f in sorted(os.listdir(path)):
        if 'run' not in f:
            continue
        if mode == 'bold' and 'desc-raw' not in f:
            continue
        if mode == 'ca2' and 'desc-preproc' not in f:
            continue
        data = np.load(pjoin(path, f))
        save_file = pjoin(tgt, mode, f"{f.split('.')[0]}.mat")
        sio.savemat(save_file, dict(data=data))
        shapes[mode].append(data.shape)

In [4]:
shapes_count = {
    k: collections.Counter(v)
    for k, v in shapes.items()
}
shapes_count

{
    'bold': Counter({(6015, 600): 206}),
    'ca2': Counter({(540, 6000): 188, (540, 5999): 13, (540, 5200): 1})
}